# BigQuery - BQML

This notebook uses BigQuery ML to train a model to predict the value of handwritten digits in BigQuery table `<PROJECT_ID>.digits.digits_prepped`.  This model is then evaluated and used for prediction using BigQuery directly.

**Prerequisites**
- `00 - Initial Setup`
- `01 - BigQuery - Data`

**Overview**

<img src="architectures/statmike-mlops-02.png">

---
## Setup

Parameters:

In [1]:
PROJECT_ID = "statmike-mlops"
REGION = 'us-central1'

---
## Train Model

Use BigQuery ML to train multiclass logistic regression model:

In [2]:
%%bigquery
CREATE OR REPLACE MODEL `digits.digits_lr`
OPTIONS
  ( model_type='LOGISTIC_REG',
    auto_class_weights=TRUE,
    input_label_cols=['target']
  ) AS
SELECT * EXCEPT(splits,target_OE)
FROM `digits.digits_prepped`
WHERE splits = 'TRAIN'

Query complete after 0.01s: 100%|██████████| 3/3 [00:00<00:00, 1412.38query/s]                        


""


Review the iterations from training:

In [3]:
%%bigquery
SELECT *
FROM ML.TRAINING_INFO(MODEL `digits.digits_lr`)
ORDER BY iteration

Downloading: 100%|██████████| 13/13 [00:01<00:00, 10.63rows/s]


,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,0.194529,0.193412,0.2,4545
1,0,1,0.141332,0.139327,0.4,5692
2,0,2,0.087229,0.085955,0.8,6174
3,0,3,0.052207,0.052151,1.6,6124
4,0,4,0.032802,0.033116,3.2,6594
5,0,5,0.021702,0.021960,6.4,6219
6,0,6,0.016512,0.017956,12.8,27459
7,0,7,0.013911,0.015276,3.2,6413
8,0,8,0.012657,0.014600,6.4,6675
9,0,9,0.010975,0.013298,12.8,6301


---
## Evaluate Model

Review the model evaluation statistics on the Test/Train splits:

In [4]:
%%bigquery
SELECT 'TRAIN' as SPLIT, * FROM ML.EVALUATE (MODEL `digits.digits_lr`,
    (SELECT * FROM `digits.digits_prepped` WHERE SPLITS='TRAIN'))
UNION ALL
SELECT 'TEST' as SPLIT, * FROM ML.EVALUATE (MODEL `digits.digits_lr`,
    (SELECT * FROM `digits.digits_prepped` WHERE SPLITS='TEST'))

Downloading: 100%|██████████| 2/2 [00:01<00:00,  1.68rows/s]


,SPLIT,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,TRAIN,0.982413,0.982238,0.982444,0.982244,0.098024,0.999859
1,TEST,0.968238,0.963715,0.965147,0.965496,0.150686,0.999289


Review the confusion matrix for each split:

In [5]:
%%bigquery
SELECT *
FROM ML.CONFUSION_MATRIX (MODEL `digits.digits_lr`,(
    SELECT *
    FROM `digits.digits_prepped`
    WHERE splits = 'TRAIN')
  );

Downloading: 100%|██████████| 10/10 [00:01<00:00,  8.63rows/s]


,expected_label,_0,_1,_2,_3,_4,_5,_6,_7,_8,_9
0,0,135,0,0,0,0,0,0,0,0,0
1,1,0,136,0,0,0,0,0,0,2,5
2,2,0,0,140,0,0,0,0,0,0,0
3,3,0,0,1,153,0,0,0,0,0,0
4,4,0,0,0,0,142,0,0,1,1,1
5,5,0,1,0,0,0,135,1,0,0,1
6,6,0,1,0,0,0,0,147,0,0,0
7,7,0,0,0,0,0,0,0,145,0,1
8,8,0,4,1,0,0,2,0,0,128,0
9,9,0,0,0,1,0,1,0,0,0,138


In [6]:
%%bigquery
SELECT *
FROM ML.CONFUSION_MATRIX (MODEL `digits.digits_lr`,(
    SELECT *
    FROM `digits.digits_prepped`
    WHERE splits = 'TEST')
  );

Downloading: 100%|██████████| 10/10 [00:01<00:00,  8.45rows/s]


,expected_label,_0,_1,_2,_3,_4,_5,_6,_7,_8,_9
0,0,42,0,0,0,1,0,0,0,0,0
1,1,0,38,0,0,0,0,0,0,0,1
2,2,0,0,37,0,0,0,0,0,0,0
3,3,0,0,0,26,0,1,0,0,2,0
4,4,0,0,0,0,34,0,0,0,1,1
5,5,0,0,0,0,0,43,0,0,0,1
6,6,0,0,0,0,0,0,32,0,1,0
7,7,0,0,0,0,0,0,0,33,0,0
8,8,0,3,0,0,0,0,0,0,36,0
9,9,0,0,0,0,0,1,0,0,0,39


---
## Predictions

Create a pandas dataframe with predictions for the test data in the table:

In [7]:
%%bigquery pred
SELECT *
FROM ML.PREDICT(MODEL `digits.digits_lr`,(
    SELECT *
    FROM `digits.digits_prepped`
    WHERE splits='TEST')
  )

Downloading: 100%|██████████| 373/373 [00:01<00:00, 245.22rows/s]


Review columns from the predictions - note that the query added columns with prefix `predicted_`

In [8]:
pred.columns

Index(['predicted_target', 'predicted_target_probs', 'p0', 'p1', 'p2', 'p3',
       'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11', 'p12', 'p13', 'p14',
       'p15', 'p16', 'p17', 'p18', 'p19', 'p20', 'p21', 'p22', 'p23', 'p24',
       'p25', 'p26', 'p27', 'p28', 'p29', 'p30', 'p31', 'p32', 'p33', 'p34',
       'p35', 'p36', 'p37', 'p38', 'p39', 'p40', 'p41', 'p42', 'p43', 'p44',
       'p45', 'p46', 'p47', 'p48', 'p49', 'p50', 'p51', 'p52', 'p53', 'p54',
       'p55', 'p56', 'p57', 'p58', 'p59', 'p60', 'p61', 'p62', 'p63', 'target',
       'target_OE', 'SPLITS'],
      dtype='object')

Print the first few rows for the columns related to the actual and predicted values:

In [9]:
pred[['target', 'predicted_target', 'predicted_target_probs', 'SPLITS']].head()

,target,predicted_target,predicted_target_probs,SPLITS
0,7,7,"[{'label': 7, 'prob': 0.9867896150202703}, {'l...",TEST
1,6,6,"[{'label': 6, 'prob': 0.9909632176848034}, {'l...",TEST
2,1,1,"[{'label': 1, 'prob': 0.978251275778806}, {'la...",TEST
3,0,0,"[{'label': 0, 'prob': 0.9989142931777109}, {'l...",TEST
4,5,5,"[{'label': 5, 'prob': 0.9969414359503399}, {'l...",TEST


Notice the nested dictionary for predicted probabilities.  In BigQuery this is a Record type structure with nested fields for `label` and `prop`.  This is returned to the pandas dataframe as a nested dictionary.

The following code sorts the dictionary for the first record by `prop`:

In [10]:
[sorted(x, key = lambda x: x['label']) for x in [pred.predicted_target_probs[0]]]

[[{'label': 0, 'prob': 3.998139398637451e-05},
  {'label': 1, 'prob': 0.0002569317170001086},
  {'label': 2, 'prob': 0.00045522988090416485},
  {'label': 3, 'prob': 0.00018284007781014508},
  {'label': 4, 'prob': 0.00018626100753949817},
  {'label': 5, 'prob': 0.0002953881532143891},
  {'label': 6, 'prob': 3.0326242031523954e-06},
  {'label': 7, 'prob': 0.9867896150202703},
  {'label': 8, 'prob': 0.0006235610780928138},
  {'label': 9, 'prob': 0.011167159046979064}]]

---
## Remove Resources
- delete model `<PROJECT_ID>.digits.digits_lr`

In [1]:
from google.cloud import bigquery
bq = bigquery.Client()

In [2]:
# NOTE: This is an input for Notebook 03 - BigQuery - BQML Online Predictions 
bq.delete_model(PROJECT_ID+'.digits.digits_lr')